In [730]:
import pandas as pd
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

<h4>Transformations for non readable columns and Datetime columns</h1>

In [828]:
#Change the change field from string to datetime.
to_yymmdd = lambda dt: datetime.strptime(dt, "%y%m%d")
to_yymmddhhmmss = lambda dt: datetime.strptime(dt, "%y%m%d %H:%M:%S")
#Translate the frequency field in english
to_english=lambda freq: 'monthly' if str.strip(freq)=='POPLATEK MESICNE' else\
                        'weekly' if str.strip(freq)=='POPLATEK TYDNE' else\
                        'pertransaction' if str.strip(freq)=='POPLATEK PO OBRATU' else\
                         str.strip(freq) if len(str.strip(freq))!=0 else np.NaN 
#Translate the frequency field in english
Change_Status_Readable=lambda Status: 'CLOSED|PAYED' if str.strip(Status)=='A' else\
                                      'CLOSED|NOTPAYED' if str.strip(Status)=='B' else\
                                      'ACTIVE|NODEBT' if str.strip(Status)=='C' else\
                                      'ACTIVE|INDEBT' if str.strip(Status)=='D' else\
                                       str.strip(Status) if len(str.strip(Status))!=0 else np.NaN 

<h2>Read Account.txt and load the data</h2>

In [732]:
#Read the file with converter functions
df_accounts=pd.read_csv('../account.txt',delimiter=';',converters={'date':to_yymmdd,'frequency':to_english})
df_accounts=df_accounts.rename(columns={'date':'create_date','district_id':'branch_dist_id'})

In [733]:
print("\n*******Check Distinct Values in Changed columns************")
print('Frequency:',df_accounts.frequency.unique())
print("\n*******Check for Null Values in dataset************")
print(df_accounts.isna().sum())
print("\n*******Check for duplicate account_id in dataset************")
print('Total count:',df_accounts.account_id.shape[0])
print('Distinct Count:',df_accounts.account_id.unique().shape[0])
print("\n*******No of records accounts dataset************")
print(df_accounts.shape[0])
print("\n*******Columns in dataset************")
print(df_accounts.columns)


*******Check Distinct Values in Changed columns************
Frequency: ['monthly' 'pertransaction' 'weekly']

*******Check for Null Values in dataset************
account_id        0
branch_dist_id    0
frequency         0
create_date       0
dtype: int64

*******Check for duplicate account_id in dataset************
Total count: 4500
Distinct Count: 4500

*******No of records accounts dataset************
4500

*******Columns in dataset************
Index(['account_id', 'branch_dist_id', 'frequency', 'create_date'], dtype='object')


<h2>Read loan.txt and load the data</h2>

In [734]:
df_loan=pd.read_csv('../loan.txt',delimiter=';',converters={'date':to_yymmdd,'status':Change_Status_Readable})
df_loan=pd.concat([df_loan,df_loan.status.str.split('|',expand=True)],axis=1).drop('status',axis=1)
df_loan=df_loan.rename(columns={0:'AccountStatus',1:'PaymentStatus','date':'loan_grant_date','amount':'loan_amount','payments':'installment_amt'})

In [735]:
print("\n*******Check Distinct Values in Changed columns************")
print('Account Status:',df_loan.AccountStatus.unique())
print('Payment Status:',df_loan.PaymentStatus.unique())
print("\n*******Check for Null Values in dataset************")
print(df_loan.isna().sum())
print("\n*******Check for duplicate loan_id in dataset************")
print('Total count:',df_loan.loan_id.shape[0])
print('Distinct Count:',df_loan.loan_id.unique().shape[0])
print("\n*******No of records loan dataset************")
print(df_loan.shape[0])
print("\n*******Columns in dataset************")
print(df_loan.columns)


*******Check Distinct Values in Changed columns************
Account Status: ['CLOSED' 'ACTIVE']
Payment Status: ['NOTPAYED' 'PAYED' 'NODEBT' 'INDEBT']

*******Check for Null Values in dataset************
loan_id            0
account_id         0
loan_grant_date    0
loan_amount        0
duration           0
installment_amt    0
AccountStatus      0
PaymentStatus      0
dtype: int64

*******Check for duplicate loan_id in dataset************
Total count: 682
Distinct Count: 682

*******No of records loan dataset************
682

*******Columns in dataset************
Index(['loan_id', 'account_id', 'loan_grant_date', 'loan_amount', 'duration',
       'installment_amt', 'AccountStatus', 'PaymentStatus'],
      dtype='object')


<h2>Read client.txt and load the data</h2>

In [736]:
def get_sex_bd(birth_num):
    birth_num=str(birth_num)
    if int(birth_num[2:4])>12:
        return 'female|'+birth_num[0:2]+str(int(int(birth_num[2:4])-50)).zfill(2)+birth_num[4:6]
    else:
        return 'male|'+birth_num

In [737]:
df_client=pd.read_csv('../client.txt',delimiter=';',converters={'birth_number':get_sex_bd})#,converters={'date':to_yymmdd,'frequency':to_english})
df_client=pd.concat([df_client,df_client.birth_number.str.split('|',expand=True)],axis=1).drop('birth_number',axis=1)
df_client=df_client.rename(columns={0:'sex',1:'birthdate','district_id':'client_dist_id'})
df_client['birthdate']=df_client.birthdate.apply(lambda dt: datetime.strptime(dt,"%y%m%d")-relativedelta(years=100) if datetime.strptime(dt,"%y%m%d")>datetime.now() else datetime.strptime(dt,"%y%m%d"))

In [738]:
print("\n*******No of records in Client dataset************")
print(df_client.shape[0])
print("\n*******Columns in Client dataset************")
print(df_client.columns)


*******No of records in Client dataset************
5369

*******Columns in Client dataset************
Index(['client_id', 'client_dist_id', 'sex', 'birthdate'], dtype='object')


<h2>Read disposition.txt and load the data</h2>

In [739]:
df_disposition=pd.read_csv('../disp.txt',delimiter=';')
df_disposition=df_disposition.rename(columns={'type':'disposition_type'})
# df_client_details=df_disposition.join(df_client.set_index('client_id'),how='inner',on='client_id')

In [740]:
print("\n*******No of disposition dataset************")
print(df_disposition.shape[0])
print("\n*******Columns in disposition dataset************")
print(df_disposition.columns)
# print("\n*******No of records df_client_details dataset************")
# print(df_client_details.shape[0])
# print("\n*******Columns in df_client_details dataset************")
# print(df_client_details.columns)


*******No of disposition dataset************
5369

*******Columns in disposition dataset************
Index(['disp_id', 'client_id', 'account_id', 'disposition_type'], dtype='object')


<h2>Read transactions.txt and load the data</h2>
<h3>Function for changing catagorical variables</h3>

In [868]:
Change_Type_Readable=lambda typ: 'credit' if str.strip(typ)=='PRIJEM' else\
                                 'debit' if str.strip(typ)=='VYDAJ' else\
                                 'debit' if str.strip(typ)=='VYBER' else\
                                  str.strip(typ) if len(str.strip(typ))!=0 else np.NaN
Change_Operation_Readable=lambda oprt:  'cc debit'    if str.strip(oprt)=='VYBER KARTOU' else\
                                        'cash credit' if str.strip(oprt)=='VKLAD' else\
                                        'oth credit' if str.strip(oprt)=='PREVOD Z UCTU' else\
                                        'cash debit' if str.strip(oprt)=='VYBER' else\
                                        'oth debit' if str.strip(oprt)=='PREVOD NA UCET' else\
                                         str.strip(oprt) if len(str.strip(oprt))!=0 else np.NaN
Change_ksym_Readable=lambda ksym:   'insurance' if str.strip(ksym)=='POJISTNE' else\
                                    'stmt dt' if str.strip(ksym)=='SLUZBY' else\
                                    'interest cr' if str.strip(ksym)=='UROK' else\
                                    'negative bal fine' if str.strip(ksym)=='SANKC. UROK' else\
                                    'regular' if str.strip(ksym)=='SIPO' else\
                                    'O/Age Pension' if str.strip(ksym)=='DUCHOD' else\
                                    'loan repay' if str.strip(ksym)=='UVER' else\
                                    'leasing' if str.strip(ksym)=='LEASING' else\
                                     str.strip(ksym) if len(str.strip(ksym))!=0 else np.NaN

In [1127]:
df_transactions=pd.read_csv('../trans.txt',delimiter=';',converters={'date':to_yymmdd,'type':Change_Type_Readable,'operation':Change_Operation_Readable,'k_symbol':Change_ksym_Readable})#,'frequency':to_english})
df_transactions.operation[(df_transactions.operation.isnull()) & (df_transactions.k_symbol=='interest cr')]='interest credit'
df_transactions.amount[df_transactions.type=='debit']*=-1
df_transactions=df_transactions.rename(columns={'date':'trans_date'})

C:\Users\Lekhu\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Lekhu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Lekhu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1136]:
datetime.now()-df_transactions.trans_date.max()

Timedelta('7691 days 08:57:47.148484')

In [1128]:
print("\n*******Check Distinct Values in Changed columns************")
print('Type:',df_transactions.type.unique())
print('Operation:',df_transactions.operation.unique())
print('K_symbol:',df_transactions.k_symbol.unique())
print("\n*******Check for Null Values in dataset************")
print(df_transactions.isna().sum())
print("\n*******Check for duplicate account_id in dataset************")
print('Total count:',df_transactions.trans_id.shape[0])
print('Distinct Count:',df_transactions.trans_id.unique().shape[0])
print("\n*******No of records dataset************")
print(df_transactions.shape[0])
print("\n*******Columns in dataset************")
print(df_transactions.columns)


*******Check Distinct Values in Changed columns************
Type: ['credit' 'debit']
Operation: ['cash credit' 'oth credit' 'cash debit' 'interest credit' 'oth debit'
 'cc debit']
K_symbol: [nan 'O/Age Pension' 'interest cr' 'regular' 'stmt dt' 'insurance'
 'negative bal fine' 'loan repay']

*******Check for Null Values in dataset************
trans_id           0
account_id         0
trans_date         0
type               0
operation          0
amount             0
balance            0
k_symbol      535314
bank          782812
account       760931
dtype: int64

*******Check for duplicate account_id in dataset************
Total count: 1056320
Distinct Count: 1056320

*******No of records dataset************
1056320

*******Columns in dataset************
Index(['trans_id', 'account_id', 'trans_date', 'type', 'operation', 'amount',
       'balance', 'k_symbol', 'bank', 'account'],
      dtype='object')


<h2>Generate required features from transactions data</h2>
<li>number of debit and credit transactions</li>
<li>Current account balance of an account</li>
<li>number of transaction per operation type</li>
<li>number of transactions per k_symbol type</li>
<li>Frequency of transactions</li>

In [1130]:
df_final_trans=df_transactions.groupby(['account_id','type']).count().reset_index()[['account_id','type','trans_id']]\
                              .pivot(index='account_id',columns='type',values='trans_id').reset_index()
df_account_bal=df_transactions.groupby(['account_id'])\
                              .apply(lambda grp: grp.sort_values('trans_date',ascending=False)).reset_index(drop=True)\
                              .groupby('account_id').head(1)[['account_id','balance']].reset_index(drop=True)
df_operation_cnt=df_transactions.groupby(['account_id','operation'])['amount'].sum().reset_index()\
                                .pivot(index='account_id',columns='operation',values='amount').reset_index()\
                                .fillna(0)
df_ksym=df_transactions.groupby(['account_id','k_symbol'])['amount'].sum().reset_index()\
                       .pivot(index='account_id',columns='k_symbol',values='amount').reset_index()\
                       .fillna(0)
df_transactions['trans_freq']=df_transactions.sort_values(['account_id','trans_date']).groupby(['account_id'])\
                                             .apply(lambda rec: rec['trans_date']-rec.shift()['trans_date']).reset_index()\
                                             ['trans_date'].dt.days
df_trans_freq=df_transactions.groupby('account_id')['trans_freq'].mean().reset_index()
df_final_trans=df_final_trans.join(df_account_bal.set_index('account_id'),on='account_id',how='inner')\
                             .join(df_operation_cnt.set_index('account_id'),on='account_id',how='inner')\
                             .join(df_trans_freq.set_index('account_id'),on='account_id',how='inner')\
                             .join(df_ksym.set_index('account_id'),on='account_id',how='left').fillna(0)

<h2>Load demographic data</h2>

In [1005]:
df_district=pd.read_csv('../district.txt',delimiter=';')#,converters={'date':to_yymmdd,'type':Change_Type_Readable,'operation':Change_Operation_Readable,'k_symbol':Change_ksym_Readable})#,'frequency':to_english})
df_district.replace('?','0',inplace=True)
df_district.A12=df_district.A12.astype(float)
df_district.A15=df_district.A15.astype(float)

<h2>Load card details data</h2>
<h3>Generate feature number of credit cards per account</h3>

In [1007]:
df_card=pd.read_csv('../card.txt',delimiter=';',converters={'issued':to_yymmddhhmmss})
df_card=df_card.join(df_disposition.set_index('disp_id'),how='inner',on='disp_id')[['card_id','account_id','type']]\
               .groupby(['account_id','type']).count().reset_index()\
               .pivot(index='account_id',columns='type',values='card_id').reset_index()\
               .fillna(0)

In [874]:
df_order=pd.read_csv('../order.txt',delimiter=';',converters={'k_symbol':Change_ksym_Readable})#,converters={'date':to_yymmdd,'type':Change_Type_Readable,'operation':Change_Operation_Readable,'k_symbol':Change_ksym_Readable})#,'frequency':to_english})
df_order=df_order.rename(columns={'amount':'order_amt','k_symbol':'ord_k_symbol'})
# df_order.replace('?','0',inplace=True)

<h2>Join all the data</h2>

In [1131]:
df_final=df_loan.join(df_accounts.set_index('account_id'),how='inner',on='account_id')\
                .join(df_disposition[df_disposition.disposition_type=='OWNER'].set_index('account_id'),how='inner',on='account_id')\
                .join(df_client.set_index('client_id'),how='inner',on='client_id')\
                .join(df_final_trans.set_index('account_id'),how='inner',on='account_id')\
                .join(df_district.set_index('A1'),how='inner',on='client_dist_id')\
                .join(df_card.set_index('account_id'),how='left',on='account_id')
df_final.fillna(0,inplace=True)

<h2>Check the counts for each file.</h2>

In [1132]:
print('Loan acc:',df_loan.shape)
print('Tot Acc:',df_accounts.shape)
print('Disposition Data:',df_disposition.shape)
print('client Data:',df_client.shape)
print('Transaction Data:',df_final_trans.shape)
print('Demographic Data:',df_district.shape)
print('card data:',df_card.shape)
print('joined data:',df_final.shape)
print('Distinct accounts:',df_final.account_id.unique().shape[0])

Loan acc: (682, 8)
Tot Acc: (4500, 4)
Disposition Data: (5369, 4)
client Data: (5369, 4)
Transaction Data: (4500, 18)
Demographic Data: (77, 16)
card data: (892, 4)
joined data: (682, 52)
Distinct accounts: 682


<h2>Save the final data to a file.</h2>

In [1134]:
df_final.to_csv('../Consolidated.txt',sep='|',index=False)